In [14]:
import pandas as pd 
import json
import re
import math
import nltk
from config import config
import psycopg2
import psycopg2.extensions
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [15]:
#nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

In [3]:
# f = open('users.json')
# users = json.load(f)
# users = pd.DataFrame(users['users']).pop('id')
# users

In [4]:
# f = open('wishlist.json')
# wishlist = json.load(f)
# wishlist = pd.DataFrame(wishlist['products'])
# wishlist 

In [65]:
#for each product in the wishlist make request to api and get the
#products full description
f = open('mocks/wishlist-products.json')
wishlist_products = json.load(f)
wishlist_description = pd.DataFrame(wishlist_products['products'])#.pop('description')
wishlist_description.drop(['model','image', 'link', 'availability','retailer' ], axis=1, inplace=True)
wishlist_description

,id,brand,price,type,description
0,ca0d6194-4906-4dbc-82c9-e7b64a93f3c4,AMD,11299,cpu,"{'# of CPU Cores': '16', '# of Threads': '32',..."
1,f63f6d5d-e32c-4cf3-a142-32a9ad8b937b,AMD,11299,cpu,"{'# of CPU Cores': '8', '# of Threads': '16', ..."
2,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,11299,cpu,"{'# of CPU Cores': '6', '# of Threads': '12', ..."
3,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,11299,cpu,"{'# of CPU Cores': '6', '# of Threads': '12', ..."
4,00f7ad83-b478-4861-884d-be26f2225285,MSI,43999,gpu,"{'NVIDIA CUDA Cores': '10496', 'Boost Clock': ..."


In [66]:
modify = re.compile(r"\([^,{}]*\)")
def clean_data(text):
    if text == '' or not list(text):
        return math.nan
    final = [(x +"-" + y).lower() for x,y in zip(list(text), list(text.values()))]
    return "  ".join(final)

    
wishlist_description['description_cl'] = wishlist_description['description'].apply(clean_data)
wishlist_description.drop(['description'], axis=1, inplace=True)
wishlist_description.dropna(inplace=True)
wishlist_description
#w = list(wishlist_description['description'])


,id,brand,price,type,description_cl
0,ca0d6194-4906-4dbc-82c9-e7b64a93f3c4,AMD,11299,cpu,# of cpu cores-16 # of threads-32 base clock...
1,f63f6d5d-e32c-4cf3-a142-32a9ad8b937b,AMD,11299,cpu,# of cpu cores-8 # of threads-16 base clock-...
2,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,11299,cpu,# of cpu cores-6 # of threads-12 base clock-...
3,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,11299,cpu,# of cpu cores-6 # of threads-12 base clock-...
4,00f7ad83-b478-4861-884d-be26f2225285,MSI,43999,gpu,nvidia cuda cores-10496 boost clock-1.70 ghz ...


In [18]:
# wishlist_description = pd.DataFrame().from_records(wishlist_description)
# wishlist_description

In [68]:
#get all products in our database and their descriptions
f = open('mocks/products.json')
products = json.load(f)
products_description = pd.DataFrame(products['products'])
products_description.drop(['model','image', 'link', 'availability', 'price' ], axis=1, inplace=True)

def clean_data(text):
    #print(list(text))
    final = [(x +"-" + str(y)).lower() for x,y in zip(list(text), list(text.values()))]
    return "  ".join(final)
#     text = text.lower()
#     text = modify.sub("", text)
    
products_description['description_cl'] = products_description['description'].apply(clean_data)
products_description.drop(['description'], axis=1, inplace=True)
products_description

,id,brand,retailer,type,description_cl
0,ca0d6194-4906-4dbc-82c9-e7b64a93f3c4,AMD,Evetech,cpu,# of cpu cores-16 # of threads-32 base clock...
1,7b05c547ff-84h84-7h3hd832f-84839937,Intel,Evetech,cpu,# of cpu cores-6 # of threads-12 processor b...
2,7b05c566-be35-47484-9tv8t-983jend4y4hw,Intel,Evetech,cpu,# of cpu cores-4 # of threads-8 processor ba...
3,4749393-8fhf4-4829-48443-94if67e20937,Intel,Evetech,cpu,# of cpu cores-8 # of threads-16 processor b...
4,7b05c56-9373-e36367e-be35-4829209376,AMD,Evetech,cpu,# of cpu cores-8 # of threads-16 # of gpu co...
5,7b0299373-e363-65c566-be35-487e20937,AMD,Evetech,cpu,# of cpu cores-12 # of threads-24 base clock...
6,7b0520937-4829-9373-ec566-be353636,AMD,Evetech,cpu,# of cpu cores-8 # of threads-16 base clock-...
7,829367-9373-e36e2-6hj8u-be35-47b05c567,AMD,Evetech,cpu,# of cpu cores-16 # of threads-32 base clock...
8,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,Evetech,cpu,# of cpu cores-6 # of threads-12 base clock-...
9,f63f6d5d-e32c-4cf3-a142-32a9ad8b937b,AMD,Evetech,cpu,# of cpu cores-8 # of threads-16 base clock-...


In [69]:
products_description.set_index('id', inplace=True)
products_description['all_features'] = products_description['brand'] +" "+ products_description['retailer'] +" "+ products_description['description_cl']
products_description['all_features'] = products_description['all_features'].apply(lambda x: x.lower())
#products_description
vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0)
tfidf_matrix = vectorizor.fit_transform(products_description['all_features'])
similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
similarity

array([[1.        , 0.06034021, 0.0632598 , 0.05598712, 0.39553184,
        0.58354032, 0.57080766, 0.59426653, 0.42354888, 0.57043224,
        0.01761893, 0.01646501],
       [0.06034021, 1.        , 0.25479636, 0.44218542, 0.06599365,
        0.07610157, 0.07549786, 0.05775531, 0.10174515, 0.07732585,
        0.00684692, 0.02411096],
       [0.0632598 , 0.25479636, 1.        , 0.3512424 , 0.07635892,
        0.06335487, 0.08293208, 0.06054983, 0.07630185, 0.08494008,
        0.01257475, 0.01175119],
       [0.05598712, 0.44218542, 0.3512424 , 1.        , 0.06872939,
        0.04978312, 0.07953076, 0.06753454, 0.05771365, 0.08145641,
        0.02477302, 0.02315056],
       [0.39553184, 0.06599365, 0.07635892, 0.06872939, 1.        ,
        0.43536562, 0.50722907, 0.29942845, 0.6094458 , 0.57842674,
        0.01786598, 0.01669588],
       [0.58354032, 0.07610157, 0.06335487, 0.04978312, 0.43536562,
        1.        , 0.61468821, 0.37283264, 0.44024332, 0.69613626,
        0.03309538,

In [70]:
rec = pd.Series(products_description.index)
def recommend(idx, cosine=similarity):
    
    #find the index of the product in the index series
    item = rec[rec == idx].index[0] #need to error check if index exists
    
    #get array of similarities and sort them in descending order
    similarity_scores = list(enumerate(cosine[item]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    index = [i[0] for i in similarity_scores]
    
    return products_description.iloc[index]


r = recommend(wishlist_description['id'][1], similarity)
r

,brand,retailer,type,description_cl,all_features
id,,,,,
7b0520937-4829-9373-ec566-be353636,AMD,Evetech,cpu,# of cpu cores-8 # of threads-16 base clock-...,amd evetech # of cpu cores-8 # of threads-16 ...
7b0299373-e363-65c566-be35-487e20937,AMD,Evetech,cpu,# of cpu cores-12 # of threads-24 base clock...,amd evetech # of cpu cores-12 # of threads-24...
7b05c56-9373-e36367e-be35-4829209376,AMD,Evetech,cpu,# of cpu cores-8 # of threads-16 # of gpu co...,amd evetech # of cpu cores-8 # of threads-16 ...
ca0d6194-4906-4dbc-82c9-e7b64a93f3c4,AMD,Evetech,cpu,# of cpu cores-16 # of threads-32 base clock...,amd evetech # of cpu cores-16 # of threads-32...
4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,Evetech,cpu,# of cpu cores-6 # of threads-12 base clock-...,amd evetech # of cpu cores-6 # of threads-12 ...


In [71]:
#result.set_index(wishlist_description['id'].index)
wishlist_cpu = wishlist_description[wishlist_description['type'] == 'cpu']
wishlist_gpu = wishlist_description[wishlist_description['type'] == 'gpu']

def get_recommendations(df_wishlist):
    recs = []
    wishlist = list(df_wishlist['id'])
    for i, product in enumerate(wishlist):
        response = list(recommend(product).index)
        recs.append(response)
        
    return recs
    
cpus = get_recommendations(wishlist_cpu)
gpus = get_recommendations(wishlist_gpu)

result_cpu = pd.DataFrame(wishlist_cpu['id'], columns=['id','recommendations'])
result_gpu = pd.DataFrame(wishlist_gpu['id'], columns=['id','recommendations'])

result_cpu['recommendations'] = cpus
result_gpu['recommendations'] = gpus

result_cpu.set_index('id', inplace=True)
result_gpu.set_index('id', inplace=True)

result_gpu['recommendations']

id
00f7ad83-b478-4861-884d-be26f2225285    [4637r-5hr84-843-j84hrf934, 7b0299373-e363-65c...
Name: recommendations, dtype: object

In [72]:
def connect():
    try:
        params = config()

        #params = config()
        #print(host, port, password, user, database)
        # host=host, port=port, database=database, user=user, password=password
        conn = psycopg2.connect(**params)
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

        curr = conn.cursor()

    except psycopg2.DatabaseError as err:
        print(err)
    return curr, conn

cur, con = connect()

In [24]:
def update_gpu(gpu_recs):
    new_products = []
    try:

        for i, row in gpu_recs.iterrows():
            query = ("""SELECT id FROM recommendation_gpu WHERE id = %s""")
            cur.execute(query, (i,))
            product = cur.fetchone()

            if product is not None:
                query = (""" UPDATE recommendation_gpu SET products = %s WHERE id = %s""")
                cur.execute(query, (row['recommendations'], i))
                con.commit()
            else:
                new_products.append((i, row['recommendations']))

            
    except(Exception, psycopg2.DatabaseError) as err:
        print(err)
    finally:
        return new_products

In [25]:
def update_cpu(cpu_recs):
    new_products = []
    try:
        for i, row in cpu_recs.iterrows():
            query = ("""SELECT id FROM recommendation_cpu WHERE id = %s""")
            cur.execute(query, (i,))
            product = cur.fetchone()

            if product is not None:
                query = (""" UPDATE recommendation_cpu SET products = %s WHERE id = %s""")
                cur.execute(query, (row['recommendations'], i))
                con.commit()
            else:
                new_products.append((i, row['recommendations']))

            
    except(Exception, psycopg2.DatabaseError) as err:
        print(err)
    finally:
        return new_products

In [26]:
def insert_cpu(id, cpu_arr):
    query = ("""INSERT INTO recommendation_cpu (id, products) VALUES (%s, %s);""")
    cur.execute(query,(id, cpu_arr))
    con.commit()


def insert_gpu(id, gpu_arr):
    query = ("""INSERT INTO recommendation_gpu (id, products) VALUES (%s, %s);""")
    cur.execute(query,(id, gpu_arr))
    con.commit()

In [16]:
new_gpu = update_gpu(result_gpu)
new_cpu = update_cpu(result_cpu)

new_cpu

[('ca0d6194-4906-4dbc-82c9-e7b64a93f3c4',
  ['829367-9373-e36e2-6hj8u-be35-47b05c567',
   '7b0299373-e363-65c566-be35-487e20937',
   '7b0520937-4829-9373-ec566-be353636',
   'f63f6d5d-e32c-4cf3-a142-32a9ad8b937b',
   '4d7ffa24-d953-4673-ad12-ab8d12f002c4']),
 ('f63f6d5d-e32c-4cf3-a142-32a9ad8b937b',
  ['7b0520937-4829-9373-ec566-be353636',
   '7b0299373-e363-65c566-be35-487e20937',
   '7b05c56-9373-e36367e-be35-4829209376',
   'ca0d6194-4906-4dbc-82c9-e7b64a93f3c4',
   '4d7ffa24-d953-4673-ad12-ab8d12f002c4']),
 ('4d7ffa24-d953-4673-ad12-ab8d12f002c4',
  ['7b05c56-9373-e36367e-be35-4829209376',
   'f63f6d5d-e32c-4cf3-a142-32a9ad8b937b',
   '7b0520937-4829-9373-ec566-be353636',
   '7b0299373-e363-65c566-be35-487e20937',
   'ca0d6194-4906-4dbc-82c9-e7b64a93f3c4'])]

In [26]:
if new_gpu is not None:
    df_gpu = pd.DataFrame(new_gpu, columns=['id', 'recs'])
    for i, row in df_gpu.iterrows():
        insert_gpu(row['id'], row['recs'])

if new_cpu is not None:
    df_cpu = pd.DataFrame(new_cpu, columns=['id', 'recs'])
    for i, row in df_cpu.iterrows():
        insert_cpu(row['id'], row['recs'])

                                     id  \
0  ca0d6194-4906-4dbc-82c9-e7b64a93f3c4   
1  f63f6d5d-e32c-4cf3-a142-32a9ad8b937b   
2  4d7ffa24-d953-4673-ad12-ab8d12f002c4   

                                                recs  
0  [829367-9373-e36e2-6hj8u-be35-47b05c567, 7b029...  
1  [7b0520937-4829-9373-ec566-be353636, 7b0299373...  
2  [7b05c56-9373-e36367e-be35-4829209376, f63f6d5...  
ca0d6194-4906-4dbc-82c9-e7b64a93f3c4 ['829367-9373-e36e2-6hj8u-be35-47b05c567', '7b0299373-e363-65c566-be35-487e20937', '7b0520937-4829-9373-ec566-be353636', 'f63f6d5d-e32c-4cf3-a142-32a9ad8b937b', '4d7ffa24-d953-4673-ad12-ab8d12f002c4']
f63f6d5d-e32c-4cf3-a142-32a9ad8b937b ['7b0520937-4829-9373-ec566-be353636', '7b0299373-e363-65c566-be35-487e20937', '7b05c56-9373-e36367e-be35-4829209376', 'ca0d6194-4906-4dbc-82c9-e7b64a93f3c4', '4d7ffa24-d953-4673-ad12-ab8d12f002c4']
4d7ffa24-d953-4673-ad12-ab8d12f002c4 ['7b05c56-9373-e36367e-be35-4829209376', 'f63f6d5d-e32c-4cf3-a142-32a9ad8b937b', '7b0520937-4829

In [73]:
f = open('mocks/wishlist-products.json')
wishlist_products = json.load(f)
wishlist_description = pd.DataFrame(wishlist_products['products'])#.pop('description')
wishlist_description.drop(['model','image', 'link', 'availability','retailer' ], axis=1, inplace=True)
wishlist_description.dropna(inplace=True)
wishlist_description

,id,brand,price,type,description
0,ca0d6194-4906-4dbc-82c9-e7b64a93f3c4,AMD,11299,cpu,"{'# of CPU Cores': '16', '# of Threads': '32',..."
1,f63f6d5d-e32c-4cf3-a142-32a9ad8b937b,AMD,11299,cpu,"{'# of CPU Cores': '8', '# of Threads': '16', ..."
2,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,11299,cpu,"{'# of CPU Cores': '6', '# of Threads': '12', ..."
3,4d7ffa24-d953-4673-ad12-ab8d12f002c4,AMD,11299,cpu,"{'# of CPU Cores': '6', '# of Threads': '12', ..."


In [44]:
for i, row in wishlist_description.iterrows():
    item = products_description[products_description.index == row['id']]
    print(row['id'])
    if item['type'].item() == 'cpu':
#         query = (""" SELECT id FROM recommendation_cpu WHERE id = %s""")
#         cur.execute(query, (row['id'],))
#         product = cur.fetchone()
        
        product = row['id'] in result_cpu.index
        print(product)
        if product == False:
            items = list(recommend(row['id']).index)
            print(row['id'], items)
            #insert_cpu(row['product_id'], items)

    elif item['type'].item() == 'gpu':
#         query = (""" SELECT id FROM recommendation_gpu WHERE id = %s""")
#         cur.execute(query, (row['id'],))
#         product = cur.fetchone()
        product = row['id'] in result_gpu.index
        print(product)
        if product == False:
            items = list(recommend(row['id']).index)
            print(row['id'], items)
            #insert_gpu(row['product_id'], items)

ca0d6194-4906-4dbc-82c9-e7b64a93f3c4
True
f63f6d5d-e32c-4cf3-a142-32a9ad8b937b
True
4d7ffa24-d953-4673-ad12-ab8d12f002c4
True
4d7ffa24-d953-4673-ad12-ab8d12f002c4
True
00f7ad83-b478-4861-884d-be26f2225285
False
00f7ad83-b478-4861-884d-be26f2225285 ['4637r-5hr84-843-j84hrf934', '7b0299373-e363-65c566-be35-487e20937', '4749393-8fhf4-4829-48443-94if67e20937', 'f63f6d5d-e32c-4cf3-a142-32a9ad8b937b', '7b0520937-4829-9373-ec566-be353636']


In [92]:
# query = ("""SELECT id from recommendation_gpu""")
# gpus = pd.read_sql_query(query, con)
# gpus = pd.DataFrame(gpus)

# query = ("""SELECT id from recommendation_cpu""")
# cpus = pd.read_sql_query(query, con)
# table = gpus.append(cpus)
table = result_gpu.append(result_cpu)
print(table.index)
print("   "  )
print(wishlist_description['id'])
items = [i for i in table.index if i not in str(wishlist_description['id'])]  

print(items)
# items = table[table.index not in wishlist_description['id'].values]
if items is not None:
    for i in items:
        if i in result_gpu.index:
#             query = ("""DELETE FROM recommendation_gpu WHERE id = %s""")
#             cur.execute(query, (i,))
            print(i, 'gpu')
        else:
            print(i, 'cpu')
#             query = ("""DELETE FROM recommendation_cpu WHERE id = %s""")
#             cur.execute(query, (i,))
    con.commit()

Index(['00f7ad83-b478-4861-884d-be26f2225285',
       'ca0d6194-4906-4dbc-82c9-e7b64a93f3c4',
       'f63f6d5d-e32c-4cf3-a142-32a9ad8b937b',
       '4d7ffa24-d953-4673-ad12-ab8d12f002c4',
       '4d7ffa24-d953-4673-ad12-ab8d12f002c4'],
      dtype='object', name='id')
   
0    ca0d6194-4906-4dbc-82c9-e7b64a93f3c4
1    f63f6d5d-e32c-4cf3-a142-32a9ad8b937b
2    4d7ffa24-d953-4673-ad12-ab8d12f002c4
3    4d7ffa24-d953-4673-ad12-ab8d12f002c4
Name: id, dtype: object
['00f7ad83-b478-4861-884d-be26f2225285']
00f7ad83-b478-4861-884d-be26f2225285 gpu


In [18]:
# products_description = pd.DataFrame().from_records(products_description)
# tags = pd.DataFrame(products['products'])[['id','type', 'brand','retailer']]
# products_description = pd.concat((tags, products_description), axis=1)
# products_description.columns.values

In [19]:
# df_products_des = products_description[['type', 'retailer', 'brand','# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
#                                         'Default TDP / TDP', 'Processor Base Frequency','Max Turbo Frequency', 'TDP']]
# #df_products_des.drop(df_products_des.filter(regex='Cache').columns, axis=1, inplace=True)
# df_products_des

In [20]:
# df_change = df_products_des.loc[df_products_des['Base Clock'].isnull()]
# df_products_des.drop(df_change.index, inplace=True)
# df_change.drop(['Base Clock', 'Default TDP / TDP', 'Max Boost Clock'], axis=1, inplace=True)
# df_change.rename({'Processor Base Frequency': 'Base Clock', 'Max Turbo Frequency': 'Max Boost Clock'}, axis=1, inplace=True)
# df_change

In [21]:
# df_products_des.drop(['Processor Base Frequency', 'Max Turbo Frequency', 'TDP'], axis=1, inplace=True)
# df_products_des.rename({'Default TDP / TDP': 'TDP'}, axis=1, inplace=True)
# #df_products_des = pd.merge(df_products_des, df_change, how='outer')
# df_products_des

In [22]:
# df_products_des = pd.merge(df_products_des, df_change, how='outer')
# df_products_des

In [23]:
# tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
# tfidf_matrix = tf.fit_transform(ds['description'])